## Akhilesh Pant (AU FTCA: MCA)

## AI Image Generator

In [5]:
import requests
import time
import threading

# Hugging Face API Key (Replace with your own key)
api_key = "hf**********************"

# Hugging Face API URL
api_url = "https://api-inference.huggingface.co/models/stabilityai/stable-diffusion-2-1"

# Headers for authentication
headers = {"Authorization": f"Bearer {api_key}"}

# User prompt for image generation
prompt = input("🚀 Enter your prompt: ").strip()

# Validate user input
if not prompt:
    print("\n🚨 Error: Prompt cannot be empty!")
    exit()

# Payload for the request
payload = {"inputs": prompt}

# Use threading.Event() for safer thread stopping
image_generated = threading.Event()

# Progress indicator function
def show_loading():
    chars = ["|", "/", "-", "\\"]
    i = 0
    while not image_generated.is_set():
        print(f"\r⏳ Generating Image... {chars[i % len(chars)]}", end="", flush=True)
        i += 1
        time.sleep(0.1)  # Faster animation

# Start the progress indicator in a separate thread
loading_thread = threading.Thread(target=show_loading)
loading_thread.start()

# Send the request
try:
    response = requests.post(api_url, headers=headers, json=payload, stream=True)

    if response.status_code == 200:
        image_path = "generated_image.png"
        with open(image_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)

        image_generated.set()  # Stop the loading animation
        print(f"\n✅ Image generated successfully and saved as {image_path}")

    elif response.status_code == 401:
        print("\n❌ Authentication Error: Check your API key.")
    elif response.status_code == 429:
        # Extract retry wait time if available
        wait_time = response.headers.get("Retry-After", "a few seconds")
        print(f"\n⏳ Rate Limit Exceeded. Please retry after {wait_time}.")
    else:
        try:
            error_message = response.json().get("error", "Unknown error occurred")
        except ValueError:
            error_message = response.text  # In case it's not JSON

        print(f"\n❌ API Error ({response.status_code}): {error_message}")

except requests.exceptions.RequestException as e:
    print(f"\n🚨 Network Error: {e}")

finally:
    image_generated.set()  # Ensure loading animation stops
    loading_thread.join()


🚀 Enter your prompt:  US President and Indian Prime Minister Meeting In New Delhi.


⏳ Generating Image... \
✅ Image generated successfully and saved as generated_image.png
